# Day 9: Rope Bridge

## Part 1

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

```
== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
```

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

```
..##..
...##.
.####.
....#.
s###..
```

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [1]:
# Loading in rope motions from input file
with open('input.txt', 'r') as f:
    rope_motions = f.read().splitlines()

In [33]:
def check_vicinity(x_head, y_head, x_tail, y_tail):
    '''
    Checks if the head is in the vicinity of the tail
    
    Inputs:
        - x_head (int): The x-coordinate of the head
        - y_head (int): The y-coordinate of the head
        - x_tail (int): The x-coordinate of the tail
        - y_tail (int): The y-coordinate of the tail
        
    Returns:
        - is_in_vicinity (bool): True if is in the vicinity; False if not in the vicinity
    '''
    # Instantiating a value to represent the vicinity boolean
    is_in_vicinity = False
    
    # Iterating over a 3x3 grid centered over the tail position
    for x in range(x_tail - 1, x_tail + 2):
        
        # Breaking loop if boolean set in nested for loop
        if is_in_vicinity:
            break
            
        for y in range(y_tail -1, y_tail + 2):
            
            # Checking if the x- and y-coordinate positions match the head
            if x == x_head and y == y_head:
                
                # Setting the boolean to be true in the case where a match is found
                is_in_vicinity = True
                
                # Breaking the loop
                break
            else:
                # Setting the boolean to be true in the case where a match is not found
                is_in_vicinity = False
                
    return is_in_vicinity

In [34]:
def move_tail(x_head, y_head, x_tail, y_tail, direction):
    '''
    Moves the position of the tail appropriately
    
    Inputs:
        - x_head (int): The x-coordinate position of the head
        - y_head (int): The y-coordinate position of the head
        - x_tail (int): The x-coordinate position of the tail
        - y_tail (int): The y-coordinate position of the tail
        - direction (str): Which direction the tail needs to head
    Returns:
        - new_x_tail (int): The new x-coordinate position of the tail
        - new_y_tail (int): The new y-coordinate position of the tail
    '''
    
    # Handing scenario where the direction is up
    if direction == 'U':
        
        # Getting the new y-coordinate
        new_y_tail = y_tail + 1
        
        # Getting the appropriate new x-coordinate
        if (x_tail - 1) == x_head:
            new_x_tail = x_tail - 1
        elif x_tail == x_head:
            new_x_tail = x_tail
        elif (x_tail + 1) == x_head:
            new_x_tail = x_tail + 1
            
    # Handing scenario where the direction is down
    if direction == 'D':
        
        # Getting the new y-coordinate
        new_y_tail = y_tail - 1
        
        # Getting the appropriate new x-coordinate
        if (x_tail - 1) == x_head:
            new_x_tail = x_tail - 1
        elif x_tail == x_head:
            new_x_tail = x_tail
        elif (x_tail + 1) == x_head:
            new_x_tail = x_tail + 1
    
    # Handing scenario where the direction is left
    if direction == 'L':
        
        # Getting the new y-coordinate
        new_x_tail = x_tail - 1
        
        # Getting the appropriate new x-coordinate
        if (y_tail - 1) == y_head:
            new_y_tail = y_tail - 1
        elif y_tail == y_head:
            new_y_tail = y_tail
        elif (y_tail + 1) == y_head:
            new_y_tail = y_tail + 1
            
    # Handing scenario where the direction is right
    if direction == 'R':
        
        # Getting the new y-coordinate
        new_x_tail = x_tail + 1
        
        # Getting the appropriate new x-coordinate
        if (y_tail - 1) == y_head:
            new_y_tail = y_tail - 1
        elif y_tail == y_head:
            new_y_tail = y_tail
        elif (y_tail + 1) == y_head:
            new_y_tail = y_tail + 1
    
    return new_x_tail, new_y_tail

In [35]:
def move_head(x_head, y_head, direction):
    '''
    Moves the position of the head appropriately
    
    Inputs:
        - x_head (int): The x-coordinate position of the head
        - y_head (int): The y-coordinate position of the head
        - direction (str): Which direction the tail needs to head
    Returns:
        - new_x_head (int): The new x-coordinate position of the head
        - new_y_head (int): The new y-coordinate position of the head
    '''
    
    # Handing scenario where the direction is up
    if direction == 'U':
        
        # Setting the coordinate values approriately
        new_x_head = x_head
        new_y_head = y_head + 1
        
    # Handing scenario where the direction is down
    if direction == 'D':
        
        # Setting the coordinate values approriately
        new_x_head = x_head
        new_y_head = y_head - 1
        
    # Handing scenario where the direction is left
    if direction == 'L':
        
        # Setting the coordinate values approriately
        new_x_head = x_head - 1
        new_y_head = y_head
        
    # Handing scenario where the direction is right
    if direction == 'R':
        
        # Setting the coordinate values approriately
        new_x_head = x_head + 1
        new_y_head = y_head
        
    return new_x_head, new_y_head

In [43]:
# Instantiating values to represent the x- and y-coordinates of the head and tail
x_head, y_head, x_tail, y_tail = 0, 0, 0, 0

# Creating a dictionary to note all the positions visited
positions_visited = {}

# Iterating over all the rope motions
for motion in rope_motions:
    
    # Splitting the direction of the motion from number of units to move
    direction, num_units = motion.split(' ')
    
    # Going through the motions (hee hee)
    for _ in range(int(num_units)):
        
        # Moving the head to its new position
        x_head, y_head = move_head(x_head, y_head, direction)
        
        # Checking if tail is still in the vicinity of the head
        if not check_vicinity(x_head, y_head, x_tail, y_tail):
            
            # Moving the tail appropriately if not in the vicinity of the head
            x_tail, y_tail = move_tail(x_head, y_head, x_tail, y_tail, direction)

In [42]:
print(f'Head position: {x_head}, {y_head}')
print(f'Tail position: {x_tail}, {y_tail}')

Head position: 26, 155
Tail position: 26, 155


In [ ]:
positions_visited